## Requirements

In [ ]:
%pip install youtube-transcript-api openai pyonepassword sqlite_utils anthropic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Youtube transcript summarizer

In [12]:
video_id = "1iGga9jCu_E" # ID from video URL

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import anthropic
from pyonepassword import OP

op = OP()
key = op.item_get('rj2spymqicqsgohimytxnkvwpa').credential
client = anthropic.Anthropic(api_key=key)

model = "claude-3-7-sonnet-20250219"

transcript = YouTubeTranscriptApi.get_transcript(video_id) 
full_transcript = ' '.join(entry['text'] for entry in transcript) # Transcript is retuned in chunks for some reason

response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=2000,
    thinking={
        "type": "enabled",
        "budget_tokens": 1024
    },
    messages=[
        {
            "role": "user",
            "content": (
                "Summarize the transcript. If the video is about trading, summarize the trader's directional bias. "
                "Also read between the lines to determine what's ACTUALLY being said and explain. "
                "Use markdown formatting. Don't skip any asset pairs mentioned. "
                "Write a personal update about the trader at the end if he mentions anything.\n\n"
                f"Transcript: {full_transcript}"
            ),
        }
    ]
)
ai_thinking = response.content[0]
ai_message = response.content[1]

from IPython.display import display, Markdown
display(Markdown(f"### Thinking:\n\n{ai_thinking.thinking}"))
display(Markdown(f"### Response:\n\n{ai_message.text}"))

### Thinking:

I need to create a summary of the transcript focusing on Tino's directional bias for various assets while explaining what's "actually" being said (reading between the lines). I should use markdown formatting, include all asset pairs mentioned, and add a personal update about Tino at the end if mentioned.

Let me identify all the asset pairs/markets discussed:
1. Bitcoin (BTC)
2. NASDAQ
3. Nvidia (NVDA)
4. Ethereum (ETH)
5. Solana (SOL)
6. XRP
7. Euro
8. US Dollar/Dollar dominance
9. Oil
10. MicroStrategy (MSTR)
11. Bonds/2-year yield

Now I'll analyze Tino's directional bias for each and read between the lines:

### Response:

# Tino's Market Analysis Summary

## Bitcoin (BTC)
**Directional Bias:** *Cautiously bullish short-term* after hitting predicted reversal level, but still wary of further downside.

**What's Really Happening:** Tino believes the recent Bitcoin crash was manufactured by big players to induce panic selling from retail investors so institutions could accumulate at lower prices. He points to the "disgusting" bid orders and Coinbase premium index showing buying activity while retail panics. He's suggesting a potential bull trap where price might rise before another leg down.

> "They need to do something psychologically: retail's like 'Oh my God Bitcoin down Bitcoin down quickly sell sell sell I hate crypto I hate this.'"

## NASDAQ/Equity Markets
**Directional Bias:** *Neutral to bearish* - describes it as "embarrassing" and in an "XRP formation" (sideways).

**What's Really Happening:** Tino sees the market in a holding pattern due to caution around Trump's potential tariff announcements. He believes traders are waiting for Trump's statements before making significant moves, which explains the sideways action despite other indicators suggesting movement.

## Nvidia (NVDA)
**Directional Bias:** *Bearish* - notes it's "gapping down" and "not doing too well."

**What's Really Happening:** Tino mentions Nvidia is at "discount" levels, has closed a gap, and is approaching interesting price zones. He alludes to a correlation someone mentioned where if Nvidia drops to $80, Bitcoin would get "smacked."

## Ethereum (ETH)
**Directional Bias:** *Bearish fulfilled* - projected drop to 2,123 was surpassed with ETH falling to 2,274.

**What's Really Happening:** Tino is suggesting his analysis was correct about Ethereum clearing out the predicted wick, indicating his technical analysis approach is reliable.

## Solana (SOL)
**Directional Bias:** *Relatively bullish* compared to other cryptos - "the only one that's been doing relatively well."

**What's Really Happening:** Tino suspects potential market manipulation, noting that Binance denied manipulating Solana's price. He questions why SOL hasn't dropped as much as other cryptos, suggesting unusual support or buying pressure.

## XRP
**Directional Bias:** *Neutral* - "not really doing too much" at $21.10.

**What's Really Happening:** Tino uses XRP as his reference for a sideways, non-moving market, even coining the term "XRP formation" to describe the NASDAQ's behavior.

## Dollar Dominance/Euro
**Directional Bias:** *Bullish on USD* - "Dollar's going to 109."

**What's Really Happening:** Tino believes the dollar is positioning for strength, especially if Trump announces tariffs. He sees the dollar consolidating before a move higher, which would likely put pressure on other assets.

## Oil
**Directional Bias:** *Bearish* - "trading negative."

**What's Really Happening:** Tino notes the traditional inverse relationship with stocks isn't working - lower oil should boost NASDAQ but it isn't, suggesting other factors (likely political uncertainty) are overpowering normal market relationships.

## MicroStrategy (MSTR)
**Directional Bias:** *Concerned* - "they're going to be itching."

**What's Really Happening:** Tino points out that MSTR's convertible bond buyers need Bitcoin to reach much higher levels to see returns, suggesting their aggressive Bitcoin accumulation strategy is facing serious pressure.

## Bonds/Yields
**Directional Bias:** *Noting downward yield movement* as potentially positive for stocks.

**What's Really Happening:** Tino mentions the market concept that when 2-year yields hit 4%, it's traditionally a good time to buy stocks, implying a potential opportunity despite current market fear.

---

## Reading Between the Lines - Market Dynamics

Tino is describing a market where:

1. **Institutional manipulation** is driving crypto prices down to accumulate from panicking retail traders
2. **Political uncertainty** from Trump's potential tariff announcements is keeping markets cautious
3. **End-of-month effects** are contributing to volatility (noting the monthly candle structure)
4. **Short covering** may drive a temporary rise in Bitcoin before potential further drops
5. **Black Rock's ETF actions** are influencing Bitcoin prices through deposits and withdrawals

His core message is that retail traders should be extremely cautious, use "light size" for positions, and understand that what appears to be a reversal could be a trap.

---

## Personal Update

Tino mentioned he'll be "stepping away from the charts today" due to the uncertainty around Trump's upcoming statements and potential market impact. He appears frustrated with the political influence on markets, particularly noting how frequently Trump has appeared in the economic calendar this week. He'll return with a "Monday flavor" analysis after the weekend.

In [14]:
""" from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
from pyonepassword import OP

op = OP()
openai_key = op.item_get('ociiuq6gchlgrjr2fopzbndrnq').credential
client = OpenAI(api_key=openai_key)

transcript = YouTubeTranscriptApi.get_transcript(video_id) 
full_transcript = ' '.join(entry['text'] for entry in transcript) # Transcript is retuned in chunks for some reason

model = "gpt-4o"
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        { 
            "role": "system", 
            "content": "You summarize YouTube transcripts to help traders determine a directional bias and have a profitable day." 
        },
        {
            "role": "user",
            "content": (
                "Summarize the transcript and determine Tino's directional bias. "
                "Also read between the lines to determine what's ACTUALLY being said and explain. "
                "Use markdown formatting. Don't skip any asset pairs mentioned. "
                "Write a personal update about Tino at the end if he mentions anything.\n\n"
                f"Transcript: {full_transcript}"
            ),
        }
    ]
)
ai_explanation = chat_completion.choices[0].message.content

from IPython.display import display, Markdown
display(Markdown(ai_explanation)) """

' from youtube_transcript_api import YouTubeTranscriptApi\nfrom openai import OpenAI\nfrom pyonepassword import OP\n\nop = OP()\nopenai_key = op.item_get(\'ociiuq6gchlgrjr2fopzbndrnq\').credential\nclient = OpenAI(api_key=openai_key)\n\ntranscript = YouTubeTranscriptApi.get_transcript(video_id) \nfull_transcript = \' \'.join(entry[\'text\'] for entry in transcript) # Transcript is retuned in chunks for some reason\n\nmodel = "gpt-4o"\nchat_completion = client.chat.completions.create(\n    model=model,\n    messages=[\n        { \n            "role": "system", \n            "content": "You summarize YouTube transcripts to help traders determine a directional bias and have a profitable day." \n        },\n        {\n            "role": "user",\n            "content": (\n                "Summarize the transcript and determine Tino\'s directional bias. "\n                "Also read between the lines to determine what\'s ACTUALLY being said and explain. "\n                "Use markdown form

In [15]:
full_transcript

"so Bitcoin well well well you know what everyone's losing their brain right now even micro strategies there like not looking good bro today guys it's dday or not all right today we're going to know if Bitcoin has come to the level that we would been expecting from yesterday's live the important level um not yesterday's the the day before the important range that Bitcoin needed to reverse from so if you haven't watched Wednesday night's video then you won't understand what I'm talking about but I'm going to show you exactly what the story is excuse me I'm trying to approach the charts today in the most relaxed way possible mad love man um baates yeah you've been here for a while man and yeah there's a lot of people that are being rattled by this deicious okay this ain't no Sala dip man we're talking quite an aggressive drop so what I'm going to do now is just break down the logic for you okay because there's no news out there that's really sparking any interest in cryptocurrency you wa

## Database

In [17]:
from sqlite_utils import Database
from datetime import datetime

# Initialize the database
db = Database("video_data.db")
table = "video_analysis"

# Check for duplicate video_id
if not db[table].exists() or not any(row['video_id'] == video_id for row in db[table].rows):
    # Insert a new row into the video_analysis table
    db[table].insert({
        "datetime": datetime.now(),
        "video_id": video_id,
        "transcript": full_transcript,
        "model": model,
        "ai_explanation": response.content[1].text
    })
else:
    print(f"Video ID {video_id} already exists in the database.")

## Possible future work:

To enhance the morning summary for a day trader focusing on cryptocurrency, you can incorporate additional data points that provide a comprehensive view of the market. Here are some suggestions:

1. **Market Sentiment Analysis**:
   - Use sentiment analysis tools to gauge the overall market sentiment from social media platforms like Twitter, Reddit, and crypto forums.
   - Incorporate sentiment scores or trends to provide insights into the general mood of the market.

2. **Technical Indicators**:
   - Include key technical indicators such as moving averages (e.g., 50-day, 200-day), RSI (Relative Strength Index), MACD (Moving Average Convergence Divergence), and Bollinger Bands.
   - Highlight any significant technical patterns or signals (e.g., golden cross, death cross).
   - Ben Cowan (into the crypto verso): bull market support band, dominance

3. **News and Events**:
   - Summarize recent news articles or headlines that could impact the crypto market, such as regulatory changes, major partnerships, or technological advancements.
   - Include a calendar of upcoming events like earnings reports, economic data releases, or major conferences.

4. **On-Chain Data**:
   - Provide insights from on-chain data such as transaction volumes, active addresses, and large transactions (whale movements).
   - Highlight any unusual activity that could indicate market manipulation or significant interest.

5. **Volatility Index**:
   - Include a measure of market volatility, such as the Crypto Volatility Index (CVI), to help traders assess potential risk levels.

6. **Exchange Flows**:
   - Track the inflow and outflow of cryptocurrencies on major exchanges to identify potential buying or selling pressure.

7. **Correlation with Other Markets**:
   - Analyze the correlation between cryptocurrencies and other asset classes like stocks, gold, or fiat currencies to understand broader market trends.

8. **Regulatory Updates**:
   - Provide updates on any regulatory developments that could affect the crypto market, such as new laws or enforcement actions.

9. **Macro-Economic Indicators**:
   - Include relevant macroeconomic indicators such as interest rates, inflation data, and employment figures that could influence market sentiment.

10. **Community and Developer Activity**:
    - Track activity on platforms like GitHub to gauge the level of development and community engagement for specific cryptocurrencies.

By incorporating these data points, you can provide a more holistic and actionable morning summary for day traders in the crypto market.

## Experiments

In [ ]:
%pip install tradingview-ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from tradingview_ta import TA_Handler, Interval, Exchange

btc = TA_Handler(
    symbol="BTCUSDT.P",
    screener="crypto",
    exchange="binance",
    interval=Interval.INTERVAL_1_DAY
)
print("BTCUSDT Futures", btc.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

BTCUSDT Futures {'RECOMMENDATION': 'SELL', 'BUY': 3, 'SELL': 14, 'NEUTRAL': 9}
